In [1]:
# install spark
!pip install pyspark

  Using cached pyspark-3.3.2-py2.py3-none-any.whl
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)


In [2]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/04/04 21:16:49 WARN Utils: Your hostname, Fengs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.17 instead (on interface en0)
23/04/04 21:16:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 21:16:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load the flight dataset
src = '/Users/fengzheng/Desktop/Coding/Practice/CS5540Project/data_raw/flight_raw'
flight_df = spark.read.option('header','true').csv(src)

# Create timestamp column
from pyspark.sql.functions import *
flight_df = flight_df.withColumn('Date',to_date(concat_ws('-', flight_df.Year, flight_df.Month,flight_df.DayofMonth)))

# Select needed columns
flight_df = flight_df.select('Date','Airline','Origin','Dest','Cancelled','OriginStateName','DestStateName')\
                        .sort('Date')

# Print schema
flight_df.printSchema()
flight_df.show(5)

root
 |-- Date: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- DestStateName: string (nullable = true)



+----------+--------------------+------+----+---------+---------------+-------------+
|      Date|             Airline|Origin|Dest|Cancelled|OriginStateName|DestStateName|
+----------+--------------------+------+----+---------+---------------+-------------+
|2019-01-01|ExpressJet Airlin...|   IAH| BTR|    False|          Texas|    Louisiana|
|2019-01-01|ExpressJet Airlin...|   ORD| SDF|    False|       Illinois|     Kentucky|
|2019-01-01|ExpressJet Airlin...|   LFT| IAH|    False|      Louisiana|        Texas|
|2019-01-01|ExpressJet Airlin...|   IAH| VPS|     True|          Texas|      Florida|
|2019-01-01|ExpressJet Airlin...|   GPT| IAH|    False|    Mississippi|        Texas|
+----------+--------------------+------+----+---------+---------------+-------------+
only showing top 5 rows



In [13]:
# Load the covid dataset
src2 = '/Users/fengzheng/Desktop/Coding/Practice/CS5540Project/data_raw/covid_raw/'
covid_df = spark.read.option('header','true').csv(src2)

# Create timestamp column
from pyspark.sql.functions import *
covid_df = covid_df.withColumn('Date',to_date(covid_df.date))

# Select needed columns
covid_df = covid_df.select('Date','state','cases','deaths')\
                        .sort('Date')

# Print schema
covid_df.printSchema()
covid_df.show(5)

root
 |-- Date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- cases: string (nullable = true)
 |-- deaths: string (nullable = true)

+----------+----------+-----+------+
|      Date|     state|cases|deaths|
+----------+----------+-----+------+
|2020-01-21|Washington|    1|     0|
|2020-01-22|Washington|    1|     0|
|2020-01-23|Washington|    1|     0|
|2020-01-24|Washington|    1|     0|
|2020-01-24|  Illinois|    1|     0|
+----------+----------+-----+------+
only showing top 5 rows



In [14]:
# submit the cleaned dataset
flight_df.repartition(1).write.csv("/Users/fengzheng/Desktop/output")
covid_df.repartition(1).write.csv("/Users/fengzheng/Desktop/output")

[823.799s][warning][gc,alloc] Executor task launch worker for task 3.0 in stage 22.0 (TID 207): Retried waiting for GCLocker too often allocating 131074 words
[823.799s][warning][gc,alloc] Executor task launch worker for task 4.0 in stage 22.0 (TID 208): Retried waiting for GCLocker too often allocating 131074 words
[823.799s][warning][gc,alloc] Executor task launch worker for task 7.0 in stage 22.0 (TID 211): Retried waiting for GCLocker too often allocating 131074 words
23/04/04 21:30:31 ERROR Executor: Exception in task 7.0 in stage 22.0 (TID 211)
java.lang.OutOfMemoryError: Java heap space
23/04/04 21:30:31 ERROR Executor: Exception in task 3.0 in stage 22.0 (TID 207)
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:50)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.s

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/fengzheng/Desktop/Coding/Practice/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/20/f_90crdd4zdfqnkxhltvnhnm0000gn/T/ipykernel_31305/3964979932.py", line 2, in <module>
    flight_df.repartition(1).write.csv("/Users/fengzheng/Desktop/output")
  File "/Users/fengzheng/Desktop/Coding/Practice/venv/lib/python3.9/site-packages/pyspark/sql/readwriter.py", line 1240, in csv
    self._jwrite.csv(path)
  File "/Users/fengzheng/Desktop/Coding/Practice/venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/Users/fengzheng/Desktop/Coding/Practice/venv/lib/python3.9/site-packages/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/Users/fengzheng/Desktop/Coding/Practice/venv/lib/python3.9/site-packa

ConnectionRefusedError: [Errno 61] Connection refused